# Inception v1

inception net 想解決三個問題

- 不同尺寸的filter對待不同大小的對象
- 太深的網路會overfitting 梯度消失
- 將比較大的filter的convolution直接鏈結計算量太大

不同尺寸的filter在同一層

先使用1x1的convo將輸入channel壓縮減小計算量

#### InceptionV1-Inception Layer

inception Layer:：目的在於結合不同 Receptive Field的特徵圖

由於信息位置的巨大差異，為卷積操作選擇合適的卷積核大小就比較困難

信息分佈更全局性的圖像偏好較大的卷積核

信息分佈比較局部的圖像偏好較小的卷積核

Inception概念：結合不同大小的 Kernels借以獲得不同尺度的訊息。

圖中展示了經典的 Inception架構，接在 Feature Maps 後一共有四條分支，其中三條先經過 1\*1 kernel 的壓縮，這樣做的意義主要是為了控制輸出 Channels 的深度，並同時能增加模型的非線性，一條則是先通過3\*3 kernel 的 Maxpooling。

為了確保輸出 Feature Map 在長寬上擁有一樣尺寸，我們就要借用 Padding 技巧，1\*1 kernel 輸出大小與輸入相同，而 3\*3、5\*5 kernel 則分別設定補邊值為 1、2，在 tensorflow、Keras 中最快的方式就是設定padding=same，就能在步伐為1時確保輸出尺寸維持相同。

#### InceptionV1-1\*1 Convolution

1\*1 kernel 的壓縮其實就是一般的卷積，然而它的好處在於能用相當少的參數量，達到壓縮特徵圖深度的目的，舉個例子來說，當輸入 Feature Map 為 (batch_size,14,14,192)，要將其壓縮為(batch_size,14,14,n)，我們只需要 1\*1\*192\*n+n 個參數量，當然同樣事情也可以用 3\*3 kernel 達成，但參數量就會變為 3\*3\*192\*n+n。

#### InceptionV1- Auxiliary classifiers

![image](https://miro.medium.com/max/1656/1*uW81y16b-ptBDV8SIT1beQ.png)

Auxiliary classifiers 在之後的文獻中比上少看見，原論文中提到其存在的目的在於通過中間層的分類損失，監督神經網路，最後會按一個較小的權重（0.3）加到最終分類結果中，這樣相當於做了模型融合，同時給網絡增加了反向傳播的梯度信號，也提供了額外的正則化。

![Incpeiton](Inception架構.png)

# Inception v3

InceptionV2 的重點就在於引入了『Batch Normalization』，之前已有獨立篇章介紹，在這就不多加贅述。

- factorizing convo: nxn nx1 and 1xn
- 5x5 $\rightarrow$ 2個 3x3
- concat nx1 and 1xn

- 將n\*n的卷積核以1\*n+n\*1的卷積核取代
- 將1個卷積拆成2個卷積，使得網絡深度更深，增加了網絡的非線性（每增加一層都要經過Activation Function）。
- 如原本3\*3的 Kernel 就會被拆解為1\*3 + 3\*1，因此觀看右圖我們可發現 Inception Block 架構變得更深。
- 然而原文也提及，此種結構放在神經網路前幾層效果並不好，建議放到較深的層數中使用。

![image](https://cf2.ppt-online.org/files2/slide/m/MqNwvc6aRTdsno7kfxgupY59Jy8QEi0Br2UDAt4WeO/slide-8.jpg)

In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras import backend as K
from keras import layers
from keras.layers import BatchNormalization
from keras.layers import Activation

## 導入InceptionV2-有BatchNormalization的Convolution

In [2]:
def Conv2d_bn(x,filters,kernel_size,padding='same',strides=(1, 1),normalizer=True,activation='relu',name=None):
    if name is not None:
        conv_name = name + '_conv'
        bn_name = name + '_bn'
        act_name = name + '_act'
    else:
        conv_name = None
        bn_name = None
        act_name = None
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    x = Conv2D(
            filters, kernel_size,
            strides=strides, padding=padding,
            use_bias=False, name=conv_name)(x)
    if normalizer:
        x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    if activation:
        x = Activation(activation, name=act_name)(x)
    return x

## 參考上圖搭建 InceptionV1_block

In [5]:
def InceptionV1_block(x, specs,channel_axis, name):
    (br0, br1, br2, br3) = specs   # ((64,), (96,128), (16,32), (32,))
    branch_0 = Conv2d_bn(x, br0[0], (1, 1), name=name+"_Branch_0")

    branch_1 = Conv2d_bn(x, br1[0], (1, 1), name=name+"_Branch_1")
    branch_1 = Conv2d_bn(branch_1, br1[1], (3, 3), name=name+"_Branch_1_1")

    '''Branch_2'''
    branch_2 = Conv2d_bn(x, br2[0], (1, 1), name=name+"_Branch_2")
    branch_2 = Conv2d_bn(branch_2, br2[1], (5, 5), name=name+"_Branch_2_1")
    
    '''Branch_3'''
    maxpoolb3 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same', name = 'maxpoolb3')(x)
    branch_3 = Conv2d_bn(maxpoolb3, br3[0], (1, 1), name=name+"_Branch_3")


    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name=name+"_Concatenated")
    return x

## 測試

In [12]:
img_input = Input(shape=(224,224,1))
x=InceptionV1_block(img_input, ((64,), (96,128), (16,32), (32,)), 3, 'Block_1')
model = Model(img_input, x)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
Block_1_Branch_1_conv (Conv2D)  (None, 224, 224, 96) 96          input_8[0][0]                    
__________________________________________________________________________________________________
Block_1_Branch_2_conv (Conv2D)  (None, 224, 224, 16) 16          input_8[0][0]                    
__________________________________________________________________________________________________
Block_1_Branch_1_bn (BatchNorma (None, 224, 224, 96) 288         Block_1_Branch_1_conv[0][0]      
____________________________________________________________________________________________

## 將 InceptionV1_block中n*n卷積改為1 x n+n x 1

In [13]:
def InceptionV3_block(x, specs,channel_axis, name):
    (br0, br1, br2, br3) = specs   # ((64,), (96,128), (16,32), (32,))
    branch_0 = Conv2d_bn(x, br0[0], (1, 1), name=name+"_Branch_0")

    branch_1 = Conv2d_bn(x, br1[0], (1, 1), name=name+"_Branch_1")
    branch_1 = Conv2d_bn(branch_1, br1[1], (1, 3), name=name+"_Branch_1_1")
    branch_1 = Conv2d_bn(branch_1, br1[1], (3, 1), name=name+"_Branch_1_2")

    '''Branch_2'''
    branch_2 = Conv2d_bn(x, br2[0], (1, 1), name=name+"_Branch_2")
    branch_2 = Conv2d_bn(branch_2, br2[1], (1, 5), name=name+"_Branch_2_1")
    branch_2 = Conv2d_bn(branch_2, br2[1], (5, 1), name=name+"_Branch_2_2")
    
    '''Branch_3'''
    maxpoolb3 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same', name = 'maxpoolb3')(x)
    branch_3 = Conv2d_bn(maxpoolb3, br3[0], (1, 1), name=name+"_Branch_3")

    x = layers.concatenate(
        [branch_0, branch_1, branch_2, branch_3],
        axis=channel_axis,
        name=name+"_Concatenated")
    return x

## 測試

In [15]:
img_input = Input(shape=(224,224,1))
x=InceptionV3_block(img_input, ((64,), (96,128), (16,32), (32,)), 3, 'Block_1')
print(x)
model = Model(img_input, x)
model.summary()

KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 256), dtype=tf.float32, name=None), name='Block_1_Concatenated/concat:0', description="created by layer 'Block_1_Concatenated'")
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 224, 224, 1) 0                                            
__________________________________________________________________________________________________
Block_1_Branch_1_conv (Conv2D)  (None, 224, 224, 96) 96          input_10[0][0]                   
__________________________________________________________________________________________________
Block_1_Branch_2_conv (Conv2D)  (None, 224, 224, 16) 16          input_10[0][0]                   
__________________________________________________________________________________________________
Block_

## 將VGG16 Block_3中的Convolution全部改為InceptionV1_block
## Block_5中的Convolution全部改為InceptionV3_block
## 並將所有Convolution改為Conv2d_bn

#### 原vgg16架構

In [ ]:


def VGG16(include_top=True,input_tensor=None, input_shape=(224,224,1),
          pooling='max',classes=1000):
 
    img_input = Input(shape=input_shape)

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    inputs = img_input
    # Create model.
    model = Model(inputs, x, name='vgg16')

   
    return model



#### 修改後

In [7]:
def VGG16_Inception(include_top=True,input_tensor=None, input_shape=(224,224,1),
          pooling='max',classes=1000):
 
    img_input = Input(shape=input_shape)

    x = Conv2d_bn(img_input,64, (3, 3), activation='relu', padding='same', name='block1_conv1')
    x = Conv2d_bn(x,64, (3, 3), activation='relu', padding='same', name='block1_conv2')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2d_bn(x,128, (3, 3), activation='relu', padding='same', name='block2_conv1')
    x = Conv2d_bn(x,128, (3, 3), activation='relu', padding='same', name='block2_conv2')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = InceptionV1_block(x, ((64,), (96,128), (16,32), (32,)), 3, 'Block_3_1')
    x = InceptionV1_block(x, ((64,), (96,128), (16,32), (32,)), 3, 'Block_3_2')
    x = InceptionV1_block(x, ((64,), (96,128), (16,32), (32,)), 3, 'Block_3_3')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2d_bn(x,512, (3, 3), activation='relu', padding='same', name='block4_conv1')
    x = Conv2d_bn(x,512, (3, 3), activation='relu', padding='same', name='block4_conv2')
    x = Conv2d_bn(x,512, (3, 3), activation='relu', padding='same', name='block4_conv3')
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x =InceptionV3_block(x, ((128,), (192,256), (32,64), (64,)), 3, 'Block_5_1')
    x =InceptionV3_block(x, ((128,), (192,256), (32,64), (64,)), 3, 'Block_5_2')
    x =InceptionV3_block(x, ((128,), (192,256), (32,64), (64,)), 3, 'Block_5_3')
    x =MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        x = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    inputs = img_input
    # Create model.
    model = Model(inputs, x, name='vgg16')

   
    return model



In [ ]:
model = VGG16_Inception(include_top=False)

In [62]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 224, 224, 1)  0                                            
__________________________________________________________________________________________________
block1_conv1_conv (Conv2D)      (None, 224, 224, 64) 576         input_19[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 224, 224, 64) 192         block1_conv1_conv[0][0]          
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 224, 224, 64) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con